In [ ]:
!pip install pyspark

## Принято ипортировать функции как f

In [ ]:
import pyspark.sql.functions as f

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.utils import AnalysisException

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("WARN")

#### Получение списка элементов колонки

In [ ]:
data = [("John", 25), ("Alice", 30), ("Bob", 35)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.show()

name_array = [row[0] for row in df.select("Name").collect()]
print(name_array)

+-----+---+
| Name|Age|
+-----+---+
| John| 25|
|Alice| 30|
|  Bob| 35|
+-----+---+

['John', 'Alice', 'Bob']


#### Повторяем действия предыдущей главы без промежуточных переменных


In [ ]:
text = spark.read.text('/content/gdrive/MyDrive/Colab Notebooks/pyspark/ch3/files/*.txt')
print(text.show())
result = (
    text.select(f.split(f.col('value'),' ').alias('result'))
    .select(f.explode(f.col('result')).alias('result'))
    .select(f.lower(f.col('result')).alias('result'))
    .select(f.regexp_extract(f.col('result'),r'[a-z]+',0).alias('result'))
    .filter(f.col('result')!='')
    .filter(~f.col('result').isin(['is','not','the','if']))
)
print(result.show())

+--------------------+
|               value|
+--------------------+
|                    |
|The Project Guten...|
|            Melville|
|                    |
|This eBook is for...|
|no restrictions w...|
|under the terms o...|
|eBook or online a...|
|                    |
|                    |
|Title: Moby Dick;...|
|                    |
|Author: Herman Me...|
|                    |
|Release Date: Dec...|
|                2017|
|                    |
|   Language: English|
|                    |
|Character set enc...|
+--------------------+
only showing top 20 rows

None
+---------+
|   result|
+---------+
|  project|
|gutenberg|
|    ebook|
|       of|
|     moby|
|     dick|
|       or|
|    whale|
|       by|
|   herman|
| melville|
|     this|
|    ebook|
|      for|
|      use|
|       of|
|   anyone|
| anywhere|
|       at|
|       no|
+---------+
only showing top 20 rows

None


#### groupby
дает нам объект groupby, который ждет агрегации. агрегируем с помощью count  
groupby возвращает неупорядоченные данные

In [ ]:
groups = result.groupby(f.col("result"))
print(groups)
groups = groups.count()
groups.show()

GroupedData[grouping expressions: [result], value: [result: string], type: GroupBy]
+----------+-----+
|    result|count|
+----------+-----+
|    online|   24|
|      some| 1551|
|       art|  100|
|    voyage|  120|
|     those|  600|
|  currents|    8|
|       few|  382|
|     still|  728|
|    travel|   14|
| connected|   47|
|  bowsprit|    8|
|  besmoked|    1|
|     spoil|    7|
|   persist|    3|
|  blackish|    1|
|concluding|   13|
|      hope|  273|
| solemnity|   11|
|    doubts|   16|
|       fog|    6|
+----------+-----+
only showing top 20 rows



#### Сортировка orderBy
сортирует данные по выбранному столбцу. Если столбцов несколько, то сортирует сначала по первому, потом остатки по второму и тд. Можно использовать либо параметр ascending либо метод desc().  
groups.orderBy(col("count").desc())

In [ ]:
results = groups.orderBy("count", ascending=False)
results = groups.orderBy(f.col("count").desc())
results.show()

+------+-----+
|result|count|
+------+-----+
|   and|24292|
|    of|21234|
|    to|20581|
|     i|15153|
|     a|14564|
|    in|12857|
|  that| 9900|
|    it| 9451|
|   was| 8939|
|   her| 7951|
|    my| 7594|
|   you| 6722|
|   his| 6676|
|    he| 6604|
|  with| 6595|
|    as| 6502|
|   had| 5727|
|   she| 5699|
|   for| 5497|
|   but| 5481|
+------+-----+
only showing top 20 rows



#### Сохранение данных  
за счет распределенности спарка, если просто так сохранять файл, то создается папка с именем results.csv", а в ней може быть много csv файликов по несколько строчек. Чтобы регулировать количество csv файлов используется метод coalesce()  
файл перед записью можно не упорядочивать, так как при чтении спарк все равно собьет порядок

In [ ]:
results.coalesce(2).write.csv("result.csv")

#### Задания  
вернуть количество уникальных слов  
distinct() возвратит df, содержащий только уникальные объекты исходного df

In [ ]:
df = spark.createDataFrame(
    [["test", "more test", 10_000_000_000],["test", "more test", 10_000_000_000]],
     ["one", "two", "three"]
)
df.show()
df.distinct().show()

+----+---------+-----------+
| one|      two|      three|
+----+---------+-----------+
|test|more test|10000000000|
|test|more test|10000000000|
+----+---------+-----------+

+----+---------+-----------+
| one|      two|      three|
+----+---------+-----------+
|test|more test|10000000000|
+----+---------+-----------+



In [ ]:
print(results.groupby(f.col('result')).count().count())
#альтернативный способ
print(results.distinct().count())

23800
23800


вернуть 5 любых слов, которые встречаются только 1 раз

In [ ]:
results.orderBy("count").show(5)

+---------+-----+
|   result|count|
+---------+-----+
|quakerism|    1|
|      zig|    1|
| spoiling|    1|
| besmoked|    1|
|   wields|    1|
+---------+-----+
only showing top 5 rows



Используя функцию substring, верните пять самых популярных первых букв (оставьте только первую букву каждого слова).
substring(колонка, номер буквы, длина среза), первая буква - 1 а не 0

In [ ]:
(
    results
    .select(f.substring(f.col('result'),1,1).alias('let'))
    .groupby('let').count()
    .orderBy('count', ascending=False)
).show(5)

+---+-----+
|let|count|
+---+-----+
|  s| 2778|
|  c| 2238|
|  p| 1845|
|  d| 1478|
|  a| 1471|
+---+-----+
only showing top 5 rows



Вычислите количество слов, начинающихся с согласной или гласной буквы.

In [ ]:
(
    results
    .select(f.substring(f.col('result'),1,1)
    .alias('let'))
    .filter(f.col('let').isin(['a','e','i','o','u','y']))
    .count()
)

5045

поправить ошибку. они хотят столбец с count и sum

In [ ]:
results.groupby('count').count().sum()

In [ ]:
(
    results
    .groupby('count')
    .agg(f.sum('count'),f.count('count'))
).show()

+-----+----------+------------+
|count|sum(count)|count(count)|
+-----+----------+------------+
|   26|      1638|          63|
|   29|      2117|          73|
|  474|       474|           1|
|   65|      2015|          31|
|  191|       191|           1|
|  418|       418|           1|
| 1551|      1551|           1|
|  293|       586|           2|
|  222|       222|           1|
| 1127|      1127|           1|
|  442|       442|           1|
|  278|       556|           2|
|  705|       705|           1|
|  243|       243|           1|
| 3866|      3866|           1|
|   19|      2508|         132|
|   54|      1242|          23|
| 1217|      1217|           1|
|  296|       296|           1|
| 2551|      2551|           1|
+-----+----------+------------+
only showing top 20 rows

